<a href="https://colab.research.google.com/github/dipit099/Deep_Learning-Colab/blob/main/Titanic_Machine_Learning_from_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
train_data  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/titanic/train.csv')
test_data  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/titanic/test.csv')


In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.columns

In [ ]:
object_columns = train_data.select_dtypes(include=['object']).columns
print(object_columns)

In [ ]:
train_data.describe()

In [ ]:
# # Set the figure size
# plt.figure(figsize=(10, 6))

# # Create a scatter plot
# sns.scatterplot(x='Fare', y='Survived', data=train_data, hue='Survived', palette={0: 'red', 1: 'green'})

# # Set the title and labels
# plt.title('Fare vs Survived (Scatter Plot)')
# plt.xlabel('Fare')
# plt.ylabel('Survived (0 = No, 1 = Yes)')

# # Show the plot
# plt.show()


In [ ]:
train_data = train_data.drop(['Name', 'Parch', 'SibSp', 'Ticket', 'Cabin'], axis=1)

In [ ]:
print(type(train_data))

In [ ]:
# # Data Loading and Preprocessing:

imputer = SimpleImputer(strategy='mean')
train_data['Age'] = imputer.fit_transform(train_data[['Age']])

# #handle categorical data
label_encoders = {}
for column in ['Sex', 'Embarked']:
    le = LabelEncoder()
    train_data[column] = le.fit_transform(train_data[column])
    label_encoders[column] = le


In [ ]:
X_tt = train_data.drop(['PassengerId', 'Survived'], axis=1)
y_tt = train_data['Survived']



In [ ]:
# # # Normalize the data
# scaler = StandardScaler()
# X = scaler.fit_transform(X)


In [ ]:
# convert to numpy arrays...ONCE possible only
X= X_tt.values
y= y_tt.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
print(type(X_tt))
print(type(X))
print(type(X_train))

In [ ]:
# Step 4: Convert to PyTorch tensors
# Convert X features to float tensors
X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)

# Convert y labels to long tensors
y_train_tensor = torch.LongTensor(y_train)
y_test_tensor = torch.LongTensor(y_test)

# Verify lengths
len(X_train_tensor), len(X_test_tensor), len(y_train_tensor), len(y_test_tensor)

In [ ]:
# Loaded into tensor model.. otherwise u cant use tensor functions
# Create TensorDataset objects
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


len(train_loader), len(test_loader)

In [ ]:
# Model Definition
class TitanicModel(nn.Module):
    def __init__(self):
        super(TitanicModel, self).__init__()
        self.layer_1 = nn.Linear(X_train_tensor.shape[1], 128)
        self.layer_2 = nn.Linear(128, 64)
        self.layer_3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()  # using ReLU activation

    def forward(self, x):
        x = self.relu(self.layer_1(x))
        x = self.relu(self.layer_2(x))
        x = self.layer_3(x)
        return x


In [ ]:
model = TitanicModel()
criterion = nn.BCEWithLogitsLoss()      # loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)  # using Adam as optimizer and learning rate


In [ ]:
# Define the number of epochs
epochs = 200
losses = []

# Training loop
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch_X, batch_y in train_loader:
        # Forward pass
        y_pred = model(batch_X).squeeze()  # Get predicted results and remove extra dimensions

        # Measure the loss/error
        loss = criterion(y_pred, batch_y.float())  # Convert batch_y to float for BCEWithLogitsLoss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss
        total_loss += loss.item()

    # Average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    losses.append(avg_loss)

    # Print every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}')

# The loss values are stored in the 'losses' list


In [ ]:
# Evaluate the model and compare predictions with known y_test values
correct = 0
total = 0
# Initialize lists to store losses

# Ensure the model is in evaluation mode
model.eval()

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        # Forward pass
        y_pred = model(batch_X).squeeze()  # Get predicted results and remove extra dimensions

        # Apply sigmoid to convert logits to probabilities
        y_pred_prob = torch.sigmoid(y_pred)

        # Round to get the predicted class (0 or 1)
        predicted_classes = torch.round(y_pred_prob)

        # Compare with
        # Compare with actual labels
        correct += (predicted_classes.squeeze() == batch_y).sum().item()
        total += batch_y.size(0)  # Accumulate the number of samples processed

print(f'We got {correct} correct out of {total}!')
print(f'Accuracy: {correct / total:.4f}')

#we can use learning graphs for better know


In [ ]:
# Predictions

test_data = test_data.drop(['Name', 'Parch', 'SibSp', 'Ticket', 'Cabin'], axis=1)
test_data['Age'] = imputer.transform(test_data[['Age']])

for column in ['Sex', 'Embarked']:
    test_data[column] = label_encoders[column].transform(test_data[column])



In [ ]:
#dont remove rows from testdata
test_data.isnull().sum()

In [ ]:
test_data.shape

In [ ]:
test_data.head()

In [ ]:
# Prepare the test data for prediction
# Drop the 'PassengerId' column as it's not a feature
X_test = test_data.drop(['PassengerId'], axis=1)

# Save the 'PassengerId' column to use later for the output file
PassengerId = test_data['PassengerId']

# Apply the same scaler used on the training data to the test data
# X_test = scaler.transform(X_test)

# Convert the scaled test data to a PyTorch tensor
X_test_tensor = torch.FloatTensor(X_test.values)

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation for prediction
with torch.no_grad():
    # Forward pass: compute the model output for the test data
    outputs = model(X_test_tensor).squeeze()

    # Convert logits to probabilities and then to binary predictions
    predictions = torch.round(torch.sigmoid(outputs)).long()



In [ ]:
# Create a DataFrame to store the 'PassengerId' and the corresponding predictions
output_df = pd.DataFrame({
    'PassengerId': PassengerId,
    'Survived': predictions.numpy()
})

# Save the predictions to a CSV file
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/titanic/output.csv', index=False)

# Print a message indicating that the output has been saved
print('Output saved to output.csv')
